In [32]:
!wget -q https://github.com/emanhamed/Houses-dataset/archive/refs/heads/master.zip

In [33]:
!unzip -q /content/master.zip

replace Houses-dataset-master/.gitattributes? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [34]:
!pip -q install wandb

In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
import wandb
from wandb.keras import (
   WandbMetricsLogger,
   WandbModelCheckpoint,
)
run = wandb.init(project="House_price_by_numerical")
config = wandb.config
wandb_callbacks = [
   WandbMetricsLogger(log_freq=5),
   WandbModelCheckpoint("models"),
]

batch/batch_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
batch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,█▇▅▄▃▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/epoch,▁▁▂▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇▇█▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▆▇▇██
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▇▅▅▃▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_loss,█▇▅▄▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/batch_step,3995
batch/learning_rate,0.0001
batch/loss,26.94454
epoch/epoch,199


In [37]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from google.colab import drive
import numpy as np
import pandas as pd
import numpy as np
import glob
import cv2
import os
def load_house_attributes(inputPath):
	# initialize the list of column names in the CSV file and then
	# load it using Pandas
	cols = ["bedrooms", "bathrooms", "area", "zipcode", "price"]
	df = pd.read_csv(inputPath, sep=" ", header=None, names=cols)

 	# determine (1) the unique zip codes and (2) the number of data
	# points with each zip code
	zipcodes = df["zipcode"].value_counts().keys().tolist()
	counts = df["zipcode"].value_counts().tolist()
	# loop over each of the unique zip codes and their corresponding
	# count
	for (zipcode, count) in zip(zipcodes, counts):
		# the zip code counts for our housing dataset is *extremely*
		# unbalanced (some only having 1 or 2 houses per zip code)
		# so let's sanitize our data by removing any houses with less
		# than 25 houses per zip code
		if count < 25:
			idxs = df[df["zipcode"] == zipcode].index
			df.drop(idxs, inplace=True)
	# return the data frame
	return df


In [38]:
def process_house_attributes(df, train, test):
	# initialize the column names of the continuous data
	continuous = ["bedrooms", "bathrooms", "area"]
	# performin min-max scaling each continuous feature column to
	# the range [0, 1]
	cs = MinMaxScaler()
	trainContinuous = cs.fit_transform(train[continuous])
	testContinuous = cs.transform(test[continuous])
	print(type(trainContinuous))
 	# one-hot encode the zip code categorical data (by definition of
	# one-hot encoing, all output features are now in the range [0, 1])
	zipBinarizer = LabelBinarizer().fit(df["zipcode"])
	trainCategorical = zipBinarizer.transform(train["zipcode"])
	testCategorical = zipBinarizer.transform(test["zipcode"])
	# construct our training and testing data points by concatenating
	# the categorical features with the continuous features
	trainX = np.hstack([trainCategorical, trainContinuous])
	testX = np.hstack([testCategorical, testContinuous])
	# return the concatenated training and testing data
	return (trainX, testX)


In [39]:
df = load_house_attributes('/content/Houses-dataset-master/Houses Dataset/HousesInfo.txt')
(train, test) = train_test_split(df, test_size=0.25, random_state=42)
maxPrice = train["price"].max()
trainY = train["price"] / maxPrice
testY = test["price"] / maxPrice
trainX, testX = process_house_attributes(df, train, test)

<class 'numpy.ndarray'>


In [40]:
trainX.shape,trainY.shape,testX.shape,testY.shape

((271, 10), (271,), (91, 10), (91,))

In [41]:
my_model = Sequential([

    tf.keras.layers.Dense(12, input_shape=(10,),activation="relu"),
    # tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(4, activation="relu"),
    # tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation="linear"),
])

In [55]:
my_model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss = tf.keras.losses.mean_absolute_percentage_error
              )

checkpoint = tf.keras.callbacks.ModelCheckpoint("/content/drive/MyDrive/best_model_house_price_numerical",
                                            save_best_only=True)
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50)

history = my_model.fit(
    x=trainX, y=trainY,
	  validation_data=(testX, testY),
    epochs=100 ,
    callbacks=[stop_early, checkpoint, wandb_callbacks]
    )

Epoch 1/100
1/9 [==>...........................] - ETA: 5s - loss: 29.7790

wandb: Adding directory to artifact (./models)... Done. 0.0s


9/9 [==============================] - 2s 170ms/step - loss: 26.5383 - val_loss: 29.2680
Epoch 2/100
1/9 [==>...........................] - ETA: 0s - loss: 25.8688

wandb: Adding directory to artifact (./models)... Done. 0.0s


9/9 [==============================] - 2s 193ms/step - loss: 26.4807 - val_loss: 29.2498
Epoch 3/100
1/9 [==>...........................] - ETA: 0s - loss: 25.6591

...

9/9 [==============================] - 1s 95ms/step - loss: 25.5006 - val_loss: 28.2606
Epoch 99/100
1/9 [==>...........................] - ETA: 0s - loss: 32.3399

wandb: Adding directory to artifact (./models)... Done. 0.0s


9/9 [==============================] - 2s 197ms/step - loss: 25.4804 - val_loss: 28.2502
Epoch 100/100
1/9 [==>...........................] - ETA: 0s - loss: 27.1373

wandb: Adding directory to artifact (./models)... Done. 0.0s


9/9 [==============================] - 2s 212ms/step - loss: 25.4789 - val_loss: 28.2504


In [56]:
import locale

# make predictions on the testing data
print("[INFO] predicting house prices...")
preds = my_model.predict(testX)
# compute the difference between the *predicted* house prices and the
# *actual* house prices, then compute the percentage difference and
# the absolute percentage difference
diff = preds.flatten() - testY
percentDiff = (diff / testY) * 100
absPercentDiff = np.abs(percentDiff)
# compute the mean and standard deviation of the absolute percentage
# difference
mean = np.mean(absPercentDiff)
std = np.std(absPercentDiff)
# finally, show some statistics on our model
locale.setlocale(locale.LC_ALL, "en_US.UTF-8")
print("[INFO] avg. house price: {}, std house price: {}".format(
	locale.currency(df["price"].mean(), grouping=True),
	locale.currency(df["price"].std(), grouping=True)))
print("[INFO] mean: {:.2f}%, std: {:.2f}%".format(mean, std))

[INFO] predicting house prices...
3/3 [==============================] - 0s 4ms/step
[INFO] avg. house price: $533,388.27, std house price: $493,403.08
[INFO] mean: 28.25%, std: 26.49%


In [57]:
loaded_model = tf.keras.models.load_model('/content/drive/MyDrive/best_model_house_price_numerical')

loaded_model.evaluate(testX, testY)

3/3 [==============================] - 0s 4ms/step - loss: 28.2502


28.250165939331055

In [58]:
drive.mount('/content/drive', force_remount=True)
loaded_model.save('/content/drive/MyDrive/dataset/numerical_House_price.h5')

Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [59]:
loaded_model = tf.keras.models.load_model('/content/drive/MyDrive/dataset/numerical_House_price.h5')

loaded_model.evaluate(testX, testY)

3/3 [==============================] - 0s 5ms/step - loss: 28.2502


28.250165939331055

In [60]:
def predict(bedrooms, bathrooms, area, zipcode):
  bed_bath_area=abs((np.array([bedrooms,bathrooms,area])-np.array([  1.  , 1., 701.]))/(np.array([1.0e+01 ,6.5e+00 ,7.0e+03])-np.array([  1.  , 1. ,701.])))
  zipBinarizer = LabelBinarizer().fit([91901, 92276, 92677, 92880, 93446, 93510, 94501])
  inputCategorical = zipBinarizer.transform([zipcode])
  inputs = np.hstack([list(inputCategorical.ravel()), list(bed_bath_area)])

  num_data=inputs.reshape(1,10)
  print(num_data)

  preds = loaded_model.predict(num_data)
  return preds
  # diff = preds.flatten() - testY
  # percentDiff = (diff / testY) * 100
  # absPercentDiff = np.abs(percentDiff)

predict(2,1,2690,92276)*5858000

[[0.         1.         0.         0.         0.         0.
  0.         0.11111111 0.         0.31576441]]
1/1 [==============================] - 0s 66ms/step


array([[169271.17632329]])